In [330]:
from tqdm import tqdm_notebook
import os
import sys
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json

In [350]:
df = pd.read_csv('./project_data/hanriver_park.csv', encoding='utf-8' )
df

,Unnamed: 0,lat,lon
0,0,37.526337,126.933597
1,1,37.526337,126.933597
2,2,37.542360,126.901817
3,3,37.507687,126.992702
4,4,37.529346,127.074006
5,5,37.562839,126.885441
6,6,37.517630,127.086763
7,7,37.552343,126.899887
8,8,37.517835,126.970798
9,9,37.528317,126.929425


In [351]:
print(df.columns)
coordinate_list =[]
for i in range(len(df)) :
    lon = df['lon'][i]
    lat = df['lat'][i]
    coordinate = str(lon) + ',' + str(lat)
    coordinate_list.append(coordinate)
    print(str(i)+'\t'+str(lon)+'\t'+ str(lat))

Index(['Unnamed: 0', 'lat', 'lon'], dtype='object')
0	126.93359679999999	37.5263366
1	126.93359679999999	37.5263366
2	126.90181670000001	37.5423604
3	126.99270209999999	37.507687
4	127.07400600000001	37.529346000000004
5	126.8854406	37.562839399999994
6	127.08676329999999	37.5176297
7	126.8998872	37.552343
8	126.9707975	37.5178349
9	126.929425	37.528317
10	127.1222742	37.5499819
11	126.81522379999998	37.587796999999995
12	126.98606000000001	37.5004776
13	126.93359679999999	37.5263366
14	127.1225831	37.5554609
15	127.009878	37.519514
16	126.902274	37.538425


In [352]:
client_id = "mBvaixbFoTsoinOJvYyX"
client_secret = "RsgWab9vxl"
location_dic = {}
gu_dic = {}
error_index = []
i = 0

for coord in tqdm_notebook(coordinate_list):   
    try:
        encText = urllib.parse.quote(coord)
        url = "https://openapi.naver.com/v1/map/reversegeocode?encoding=utf-8&coordType=latlng&query=" + encText # json 결과, 좌표->주소 api
        # url = "https://openapi.naver.com/v1/map/geocode.xml?query=" + encText # xml 결과
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        response = urllib.request.urlopen(request)
        rescode = response.getcode()

        if(rescode==200):
            response_body = response.read()
            json_data = response_body.decode('utf-8') # 리턴값이 json
            location = json.JSONDecoder().decode(json_data) # json 파일 파이썬으로 불러오기 , json.decode()
            addr = location['result']['items'][-1]['addrdetail']['dongmyun']
            gu = location['result']['items'][-1]['addrdetail']['sigugun']
            
            gu_dic[addr] = gu
            
            if addr in list(location_dic.keys()):
                location_dic[addr] += 1
            else :
                location_dic[addr] = 1

            print(addr, gu, str(i))
        else:
            print("Error Code:" + rescode)
    
    except Exception as e :
        print('Not Found ' + addr + str(i))
        error_index.append(i)
    i = i+1

여의동 영등포구 0
여의동 영등포구 1
당산2동 영등포구 2
반포2동 서초구 3
자양3동 광진구 4
상암동 마포구 5
잠실2동 송파구 6
망원1동 마포구 7
이촌1동 용산구 8
여의동 영등포구 9
암사2동 강동구 10
방화3동 강서구 11
반포본동 서초구 12
여의동 영등포구 13
암사2동 강동구 14
잠원동 서초구 15
당산2동 영등포구 16



In [324]:
error_index

[]

In [353]:
print(len(df)); print(sum(location_dic.values()))

17
17


In [354]:
location_df = pd.DataFrame([location_dic], index = ['한강공원개수']).transpose() # dataframe은 리스트형태로, transpose

In [355]:
location_df

,한강공원개수
당산2동,2
망원1동,1
반포2동,1
반포본동,1
방화3동,1
상암동,1
암사2동,2
여의동,4
이촌1동,1
자양3동,1


In [344]:
gu_df = pd.DataFrame([gu_dic], index = ['구']).transpose()

In [345]:
gu_df

,구
고척1동,구로구
공릉2동,노원구
광장동,광진구
군자동,광진구
남가좌2동,서대문구
논현2동,강남구
대학동,관악구
대흥동,마포구
면목3.8동,중랑구
미아동,강북구


In [346]:
complete_df = pd.merge(location_df, gu_df, left_on = location_df.index, right_on = gu_df.index, left_index=True)

In [347]:
del complete_df['key_0']

In [348]:
complete_df

,대학교개수,구
고척1동,1,구로구
공릉2동,2,노원구
광장동,1,광진구
군자동,1,광진구
남가좌2동,1,서대문구
논현2동,1,강남구
대학동,1,관악구
대흥동,1,마포구
면목3.8동,1,중랑구
미아동,2,강북구


In [349]:
    complete_df.to_csv('./project_data/covert_coor_to_dong/univ_dong_final.csv' ,sep=',', encoding='utf-8')